In [51]:
# import all libraries 
import numpy as np
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.manifold import TSNE
import random
import warnings
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, f1_score
import community as comm
import community.community_louvain
import networkx as nx
from gensim.models import Word2Vec
import random
import time


warnings.filterwarnings('ignore')

In [52]:
def load_graph():
    #graphfile = 'facebook_combined.txt'
    graphfile = 'facebook_combined.txt'
    print("data file used is "+ graphfile)
    # labelfile = 'facebook_combined.nodes.labels'
    G = nx.read_edgelist(graphfile, nodetype=None)
    G = G.to_directed()
    print("Number of nodes: ", G.number_of_nodes())
    print("Number of edges: ", G.number_of_edges())
    return G

def get_embedding(G, walks, embed_size=128, window_size=5, workers=3, iter=5, **kwargs):
    kwargs["sentences"] = walks
    kwargs["min_count"] = kwargs.get("min_count", 0)
    kwargs["vector_size"] = embed_size
    kwargs["sg"] = 1  # skip gram
    kwargs["hs"] = 1  # deepwalk use Hierarchical Softmax
    kwargs["workers"] = workers
    kwargs["window"] = window_size
    kwargs["epochs"] = iter

    print("Learning embedding vectors...")
    model = Word2Vec(**kwargs)
    print("Learning embedding vectors done!")
    print("model is \n")
    print(model)
    embeddings = {}
    for word in G.nodes():
        embeddings[word] = model.wv[word]
    return embeddings


def deepwalk_walks(G, num_walks, walk_length,):
        nodes = G.nodes()
        walks = []
        for _ in range(num_walks):
            for v in nodes:
                walk = [v]
                while len(walk) < walk_length:
                    cur = walk[-1]
                    cur_nbrs = list(G.neighbors(cur))
                    if len(cur_nbrs) > 0:
                        walk.append(random.choice(cur_nbrs))
                    else:
                        break
                walks.append(walk)
        return walks


In [55]:
G = load_graph()
l = 1
walks_deepwalk = deepwalk_walks(G, walk_length=l, num_walks=80)
embeddings_deepwalk = get_embedding(G,walks_deepwalk)
G1 = G.to_undirected()

points = [[0] for i in range(G1.number_of_nodes()-1)]
for i in range(0, G1.number_of_nodes()-1):
    points[i] = embeddings_deepwalk[str(i+1)]
points = np.array(points)

print("shape of points is "+ str(points.shape))


#modularity_scores = cluster_eval(G1, embeddings_deepwalk)
#points = np.array([ [9.0, 10.0], [1000, 9000],[1,2],[2,1],[2,2],[9,9],[1000,8000]])
#k = 3

max_iters = 10
NOC = 100

start_time = time.time()
print("\nStarting kmeans...")

for k in range(99, NOC):
    clusters = KMeans(n_clusters=k, random_state=0).fit(points)
   

print("\nFinished kmeans...")
end_time = time.time()

elapsed_time = end_time - start_time
print("Execution time:", elapsed_time, "seconds")

data file used is facebook_combined.txt
Number of nodes:  4039
Number of edges:  176468
Learning embedding vectors...
Learning embedding vectors done!
model is 

Word2Vec(vocab=4039, vector_size=128, alpha=0.025)
shape of points is (4038, 128)

Starting kmeans...

Finished kmeans...
Execution time: 1.356048822402954 seconds
